<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_online_0616/blob/main/tibame_night_20250708.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


```
HTTP代碼
1. 2: 成功
2. 3: 轉址
3. 4: 錯誤
403 Forbidden
1. IP被ban: 換ip/等
2. 模仿不夠像: 缺少必要headers
```



In [10]:
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1751962851.A.45A.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")
resp = req.urlopen(r)
content = resp.read()
html = bs.BeautifulSoup(content)
# find/find_all
# .text .get_text() ["href"]

# 作者/標題/時間/看板
metas = html.find_all("span", {"class":"article-meta-value"})
author = metas[0]
board = metas[1]
title = metas[2]
post_time = metas[3]
pushes = html.find_all("div", {"class":"push"})


author_text = author.text
board_text = board.text
title_text = title.text
post_time_text = post_time.text

push_list = []
for p in pushes:
    push_meta = p.find_all("span")
    push_trans = {"推":1, "噓":-1, "→":0}
    push_data = {
        "type":push_trans[push_meta[0].text.strip()],
        "uid":push_meta[1].text.strip(),
        "text":push_meta[2].text.strip().replace(": ", ""),
        "ipdatetime":push_meta[3].text.strip()
    }
    push_list.append(push_data)


row = {
    "author":author_text,
    "title":title_text,
    "post_time":post_time_text,
    "board":board_text,
    "pushes":push_list,
}
print(row)

{'author': 'a0802459 (王月可可)', 'title': '[正妹] 短髮李珠珢', 'post_time': 'Tue Jul  8 16:20:49 2025', 'board': 'Beauty', 'pushes': [{'type': 1, 'uid': 'hyperfrog', 'text': '身材很好 但短髮有點莎味', 'ipdatetime': '42.72.86.237 07/08 16:23'}, {'type': -1, 'uid': 'kenslc199', 'text': '越看越像izsasa', 'ipdatetime': '106.64.97.99 07/08 16:26'}, {'type': 1, 'uid': 'haircc', 'text': '鷰尾', 'ipdatetime': '116.59.85.235 07/08 16:29'}, {'type': 1, 'uid': 'njnjy', 'text': '珠珠寶貝', 'ipdatetime': '36.233.28.49 07/08 16:33'}, {'type': 1, 'uid': 'loloman', 'text': '真的很漂亮', 'ipdatetime': '111.71.215.198 07/08 16:35'}, {'type': 1, 'uid': 'thewtf', 'text': '我老婆', 'ipdatetime': '49.217.198.113 07/08 16:38'}, {'type': 1, 'uid': 'Krishna', 'text': '害我覺得莎莎愈看愈可愛', 'ipdatetime': '49.217.135.204 07/08 16:45'}, {'type': 1, 'uid': 'CavendishJr', 'text': '100%的愛莉莎莎', 'ipdatetime': '27.52.58.37 07/08 16:50'}, {'type': 0, 'uid': 'CavendishJr', 'text': '莎莎本來就可愛', 'ipdatetime': '27.52.58.37 07/08 16:50'}, {'type': 1, 'uid': 'ginlom', 'text

In [ ]:
""